In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
print('current working directory: {}'.format(os.getcwd()))

current working directory: C:\Users\agarw\Dropbox\Kaggle data-science-bowl 2019\Prarit-data-science-bowl-2019


In [3]:
print('files in cwd: {}'.format(os.listdir()))

files in cwd: ['.git', '.ipynb_checkpoints', 'Feature Engineering Speed Up.ipynb', 'Feature Engineering.ipynb', 'Initial EDA.ipynb', 'sample_submission.csv', 'specs.csv', 'test.csv', 'train.csv', 'train_features.csv', 'train_labels.csv']
